In [ ]:
class ModelSelector:
  def __init__(self, dictionary, vectorizer, lib, df=df, 
               limit=10, start=2, step=1, vocab=vocab, words=words, 
               id2word=id2word):
    
    self.df = df
    self.limit = limit
    self.start = start
    self.step = step
    self.words = words
    self.dictionary = dictionary
    self.vectorizer = vectorizer
    self.lib = lib
    self.vocab = vocab
    self.words = words
    self.id2word = id2word
    self.coherence_list = []
    self.model_list = []
    self.perplexity_list = []
    self.log_likelihood_list = []
    
  def __call__(self):
    
    
    for num_topics in range(self.start, self.limit, self.step):
      lda_modeller = LDAmodeller(df = self.df, vectorizer = self.vectorizer, lib = self.lib, num_topics = num_topics, verbose = False)
      lda_modeller.train_lda()
      self.model_list.append(lda_modeller.lda_model)
      
      topics = []
      for i in range(lda_modeller.lda_model.n_components):
        topic_words = [self.vocab[j] for j in lda_modeller.lda_model.components_[i].argsort()[:-10 - 1:-1]]
        topics.append(topic_words)
        
      coherencemodel = CoherenceModel(topics=topics, texts=self.words, dictionary=self.dictionary, coherence='c_v')
      self.coherence_list.append(coherencemodel.get_coherence())
      
      perplexity = np.log(lda_modeller.lda_model.perplexity(lda_modeller.vectorized_sentences))
      self.perplexity_list.append(perplexity)
      
      log_likelihood = lda_modeller.lda_model.score(lda_modeller.vectorized_sentences)
      self.log_likelihood_list.append(log_likelihood)
    
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    
    x = range(self.start, self.limit, self.step)
    
    axs[0].plot(x, self.coherence_list)
    axs[0].set_title('Coherence')
    axs[0].set_xlabel('Num Topics')
    axs[0].set_ylabel('Coherence Score')

    axs[1].plot(x, self.perplexity_list)
    axs[1].set_title('Perplexity')
    axs[1].set_xlabel('Num Topics')
    axs[1].set_ylabel('Perplexity Score')

    axs[2].plot(x, self.log_likelihood_list)
    axs[2].set_title('Log likelihood')
    axs[2].set_xlabel('Num Topics')
    axs[2].set_ylabel('Log-likelihood Score')
    
    return model_list, coherence_values, perplexity_list, log_likelihood_scores